ok so let's think about how to go after building this KNN 

This is quite similar to the spotify example, where we took the spotify songs data and positioned a new song in that space, and found the K nearest songs. That's great but in this case we need to:
1) make sure that the new point (ideal country) that we position in space has features that are scaled correctly
2) find a way to weight the distance on different axes to a different extent


EXTRA: it would actually be really cool to have an alternative interface where instead of selecting "good" "mild" "bad" ecc... preferences, the user selects preferences like "my ideal country would have the climate of spain, with the conenction speed of lithuania, and the safety of the UAE.

In [28]:
#let's code from scratch as if it was a new project

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [29]:
df = pd.read_csv("../raw_data/merged_country_level/merged_dataset_with_knn.csv")#.set_index("Unnamed: 0")
df.rename(columns={"Unnamed: 0": "Country"}, inplace=True)
df.head(30)

,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index
0,afghanistan,960.545000,18.1,3.880000,24.9,24.24
1,albania,518.916429,22.2,81.410000,55.3,48.21
2,algeria,356.045500,22.8,16.540000,47.4,54.43
3,andorra,1257.460000,11.1,113.413333,84.7,75.56
4,angola,740.635000,27.1,22.910000,33.7,36.58
5,argentina,503.731250,15.1,93.380000,36.6,68.00
6,armenia,1347.520000,18.4,60.720000,77.9,58.07
7,australia,1870.925000,12.4,80.490000,52.7,73.35
8,austria,1517.237500,14.6,100.450000,70.5,77.83
9,azerbaijan,410.863333,19.0,69.320000,68.2,48.31


In [30]:
numeric_features = df.select_dtypes(include=['float64', 'int64']).columns
sc = MinMaxScaler()
df_scaled = df.copy()
df_scaled[numeric_features] = sc.fit_transform(df[numeric_features])
df_scaled

,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index
0,afghanistan,0.111708,0.546926,0.001900,0.085627,0.006066
1,albania,0.044422,0.679612,0.228530,0.550459,0.388729
2,algeria,0.019607,0.699029,0.038907,0.429664,0.488027
3,andorra,0.156945,0.320388,0.322079,1.000000,0.825351
4,angola,0.078202,0.838188,0.057527,0.220183,0.203065
...,...,...,...,...,...,...
150,venezuela,0.052780,0.705502,0.235837,0.000000,0.236750
151,vietnam,0.047566,0.776699,0.472201,0.610092,0.598020
152,yemen,0.073334,0.711974,0.023327,0.259939,0.023148
153,zambia,0.070271,0.831715,0.055569,0.536697,0.488186


In [31]:
knn =  NearestNeighbors(n_neighbors=155, metric="euclidean")
knn.fit (df_scaled.drop(columns=["Country"]))


NearestNeighbors(metric='euclidean', n_neighbors=155)

In [32]:
ideal_country = pd.DataFrame({"average_monthly_cost_$":[1420.0],"average_yearly_temperature":[13.0],"internet_speed_mbps":[116.6],"safety_index":[50.6],"Healthcare Index":[75.7]})
ideal_country_scaled = sc.transform(ideal_country)
ideal_country_scaled

array([[0.18170925, 0.38187702, 0.33139433, 0.47859327, 0.82758621]])

In [33]:
distances, indices = knn.kneighbors (ideal_country_scaled)

close_countries = indices[0][0:]

close_countries


/Users/federico/.pyenv/versions/3.10.6/envs/project_ics/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


array([ 15, 146,   7,  89,  50, 148, 100, 131,  41,  80,  81,  33,  73,
        53,  28,   5,  77,   8,  66, 104,  82,  75,  71,  46, 144,  37,
       109,  22, 122,  64,  44, 113, 123,  99,  34,  90,  25,  70,  18,
       135, 101, 116, 127, 126,  14,  52, 120, 112, 103, 141,  13,  98,
        78, 149,  62, 107, 151, 110,  47,  88,   3,  84,  86,  58, 147,
        69,  43,  72,  45,  38, 132,  40,  49,  26,   6,  65,   1, 111,
        27, 115,   2,  36,  92,   9,  61,  21, 140,  20,  35,  94,  97,
       125,  16,  17,  67, 117,  19,  60,  51, 128, 153, 139, 138, 134,
        93, 106,  87, 142,  68,  11, 143,  29,  31,  42, 137,  59, 119,
       154,  23,  10, 130,  85,  56,  96,  74,  79, 118,  24, 129, 105,
        54, 136, 102,  39,  95,  63,  12,  76,  83, 114, 150, 124, 133,
         4,  57,  55,  48, 152,  91,   0, 121, 145,  32,  30, 108])

In [34]:
close_countries = df_scaled.iloc[close_countries]
close_countries["country_distance_from_ideal"] = distances[0]  
close_countries #already sorted by distance


/var/folders/bp/0xr5z85d4yg8z87fl3f_x8mr0000gn/T/ipykernel_83378/2685092988.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_countries["country_distance_from_ideal"] = distances[0]


,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index,country_distance_from_ideal
15,belgium,0.181777,0.381877,0.331657,0.478593,0.828544,0.000996
146,united kingdom,0.230277,0.381877,0.387109,0.495413,0.783206,0.087838
7,australia,0.250411,0.362460,0.225840,0.510703,0.790070,0.136664
89,mexico,0.075203,0.411003,0.236013,0.417431,0.774745,0.166801
50,germany,0.174267,0.411003,0.272143,0.631498,0.768040,0.177032
...,...,...,...,...,...,...,...
121,singapore,0.488646,0.893204,1.000000,0.888379,0.766922,0.987068
145,united arab emirates,0.399761,0.990291,0.907103,0.996942,0.746648,1.012122
32,congo,0.473872,0.909385,0.102952,0.247706,0.029215,1.051910
30,congo,0.473872,0.909385,0.102952,0.247706,0.029215,1.051910


### compare different distances algorithms

In [36]:
def find_similar_countries(ideal_country, df, n_neighbors=10, distance_metric="euclidean"):
    numeric_features = df.select_dtypes(include=['float64', 'int64']).columns
    sc = MinMaxScaler()
    df_scaled = df.copy()
    df_scaled[numeric_features] = sc.fit_transform(df[numeric_features])

    knn = NearestNeighbors(n_neighbors=n_neighbors, metric=distance_metric)
    knn.fit(df_scaled.drop(columns=["Country"]))

    ideal_country_scaled = sc.transform(ideal_country)
    distances, indices = knn.kneighbors(ideal_country_scaled)

    close_countries = df_scaled.iloc[indices[0]]
    close_countries["country_distance_from_ideal"] = distances[0]
    
    return close_countries

In [38]:


find_similar_countries(ideal_country_belgium, df, n_neighbors=155, distance_metric="manhattan")

/Users/federico/.pyenv/versions/3.10.6/envs/project_ics/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/var/folders/bp/0xr5z85d4yg8z87fl3f_x8mr0000gn/T/ipykernel_83378/2815711504.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_countries["country_distance_from_ideal"] = distances[0]


,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index,country_distance_from_ideal
15,belgium,0.181777,0.381877,0.331657,0.478593,0.828544,0.001289
146,united kingdom,0.230277,0.381877,0.387109,0.495413,0.783206,0.165482
7,australia,0.250411,0.362460,0.225840,0.510703,0.790070,0.263300
50,germany,0.174267,0.411003,0.272143,0.631498,0.768040,0.308272
100,new zealand,0.189716,0.388350,0.503683,0.496942,0.714400,0.318303
...,...,...,...,...,...,...,...
121,singapore,0.488646,0.893204,1.000000,0.888379,0.766922,1.957320
145,united arab emirates,0.399761,0.990291,0.907103,0.996942,0.746648,2.001462
32,congo,0.473872,0.909385,0.102952,0.247706,0.029215,2.077371
30,congo,0.473872,0.909385,0.102952,0.247706,0.029215,2.077371


In [40]:
find_similar_countries(ideal_country_belgium, df, n_neighbors=155, distance_metric="cosine")


/Users/federico/.pyenv/versions/3.10.6/envs/project_ics/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/var/folders/bp/0xr5z85d4yg8z87fl3f_x8mr0000gn/T/ipykernel_83378/2815711504.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_countries["country_distance_from_ideal"] = distances[0]


,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index,country_distance_from_ideal
15,belgium,0.181777,0.381877,0.331657,0.478593,0.828544,1.367245e-07
146,united kingdom,0.230277,0.381877,0.387109,0.495413,0.783206,3.199113e-03
7,australia,0.250411,0.362460,0.225840,0.510703,0.790070,7.527028e-03
89,mexico,0.075203,0.411003,0.236013,0.417431,0.774745,8.613538e-03
50,germany,0.174267,0.411003,0.272143,0.631498,0.768040,1.242898e-02
...,...,...,...,...,...,...,...
32,congo,0.473872,0.909385,0.102952,0.247706,0.029215,4.752212e-01
30,congo,0.473872,0.909385,0.102952,0.247706,0.029215,4.752212e-01
152,yemen,0.073334,0.711974,0.023327,0.259939,0.023148,4.777708e-01
0,afghanistan,0.111708,0.546926,0.001900,0.085627,0.006066,5.547243e-01


In [39]:
find_similar_countries(ideal_country_belgium, df, n_neighbors=155, distance_metric="chebyshev")


/Users/federico/.pyenv/versions/3.10.6/envs/project_ics/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/var/folders/bp/0xr5z85d4yg8z87fl3f_x8mr0000gn/T/ipykernel_83378/2815711504.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  close_countries["country_distance_from_ideal"] = distances[0]


,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index,country_distance_from_ideal
15,belgium,0.181777,0.381877,0.331657,0.478593,0.828544,0.000958
146,united kingdom,0.230277,0.381877,0.387109,0.495413,0.783206,0.055715
7,australia,0.250411,0.362460,0.225840,0.510703,0.790070,0.105554
89,mexico,0.075203,0.411003,0.236013,0.417431,0.774745,0.106506
148,uruguay,0.103898,0.443366,0.441216,0.438838,0.713761,0.113825
...,...,...,...,...,...,...,...
30,congo,0.473872,0.909385,0.102952,0.247706,0.029215,0.798372
152,yemen,0.073334,0.711974,0.023327,0.259939,0.023148,0.804438
91,monaco,1.000000,0.569579,0.106031,0.877676,0.816411,0.818291
0,afghanistan,0.111708,0.546926,0.001900,0.085627,0.006066,0.821520


### Let's try to code a function and add weights to the different features

In [43]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

def find_similar_countries(
    ideal_country, 
    df, 
    n_neighbors=10, 
    distance_metric="euclidean",
    feature_weights=None
):
    """
    Find similar countries to 'ideal_country' from 'df' using KNN with optional per-feature weights.
    
    Parameters:
    -----------
    ideal_country : pd.DataFrame or np.array 
        A single-row DataFrame or 2D array with numeric columns that match those in df.
    df : pd.DataFrame
        DataFrame of countries with at least one numeric column and possibly a 'Country' column.
    n_neighbors : int
        Number of neighbors (similar countries) to return.
    distance_metric : str
        Distance metric for KNN (e.g. "euclidean", "manhattan", "minkowski", "cosine").
    feature_weights : list or np.array
        Per-feature weights. The length must match the number of numeric columns in df.
        
    Returns:
    --------
    close_countries : pd.DataFrame
        Subset of df with the closest neighbors and a 'country_distance_from_ideal' column.
    """
    # 1) Identify numeric features in df
    numeric_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
    
    # 2) Scale numeric features to [0,1] using MinMaxScaler
    sc = MinMaxScaler()
    df_scaled = df.copy()
    df_scaled[numeric_features] = sc.fit_transform(df[numeric_features])
    
    # 3) If feature_weights are provided, multiply each scaled column by its corresponding weight
    if feature_weights is not None:
        if len(feature_weights) != len(numeric_features):
            raise ValueError("Length of feature_weights must match the number of numeric features.")
        for i, col in enumerate(numeric_features):
            df_scaled[col] *= feature_weights[i]
    
    # 4) Fit NearestNeighbors on the scaled (and weighted) data
    #    Drop 'Country' column if present to avoid passing it to KNN
    knn_features = [col for col in df_scaled.columns if col in numeric_features]
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric=distance_metric)
    knn.fit(df_scaled[knn_features])
    
    # 5) Scale the ideal country with the same scaler
    #    Expecting ideal_country to be shaped like (1, num_features) or a 1-row DataFrame
    ideal_country_scaled = sc.transform(ideal_country)
    
    # 6) Apply the same weighting to the ideal country vector
    if feature_weights is not None:
        ideal_country_scaled = ideal_country_scaled * feature_weights
    
    # 7) Query the neighbors
    distances, indices = knn.kneighbors(ideal_country_scaled)
    
    # 8) Retrieve closest countries from the ORIGINAL df (not scaled),
    #    so results are more interpretable
    close_countries = df.iloc[indices[0]].copy()
    close_countries["country_distance_from_ideal"] = distances[0]
    
    return close_countries



In [54]:
ideal_country_random = pd.DataFrame({"average_monthly_cost_$":[720.0],"average_yearly_temperature":[13.0],"internet_speed_mbps":[116.6],"safety_index":[50.6],"Healthcare Index":[75.7]})


weights = [10, 1, 3, 1, 1]

closest = find_similar_countries(
    ideal_country=ideal_country_random, 
    df=df, 
    n_neighbors=155, 
    distance_metric="manhattan",
    feature_weights=weights
)

closest

/Users/federico/.pyenv/versions/3.10.6/envs/project_ics/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,Country,average_monthly_cost_$,average_yearly_temperature,internet_speed_mbps,safety_index,Healthcare Index,country_distance_from_ideal
41,ecuador,671.301000,10.8,110.440000,37.5,75.39,0.404668
89,mexico,720.950408,13.9,83.970000,46.6,72.39,0.430722
33,costa rica,789.472500,18.7,119.190000,45.9,64.29,0.567043
81,lithuania,857.710000,11.8,124.136667,67.6,74.95,0.586651
44,estonia,809.756875,10.9,92.420000,76.3,75.93,0.813395
...,...,...,...,...,...,...,...
145,united arab emirates,2851.180000,31.8,313.550000,84.5,70.63,6.181856
32,congo,3337.605000,29.3,38.450000,35.5,25.69,6.230228
30,congo,3337.605000,29.3,38.450000,35.5,25.69,6.230228
121,singapore,3434.575000,28.8,345.330000,77.4,71.90,7.123472


### more exploration, more complex function

In [29]:
# /Users/federico/code/Brgtt07/project_ics/notebooks/knn.ipynb
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import pickle
import os

def scale_dataset(df, pipeline_save_path=os.path.join('models', 'v2_scaler_pipeline.pkl')):
    """
    Scale the numeric features of the DataFrame using a pipeline and save the scaler.

    Parameters:
    -----------
    df : pd.DataFrame
        DataFrame of countries with numeric columns.
    model_path : str
        Path to save the scaler.

    Returns:
    --------
    df_scaled : pd.DataFrame
        Scaled DataFrame.
    pipeline : Pipeline
        Fitted scaling pipeline.
    """
    #df.rename(columns={"Unnamed: 0": "Country"}, inplace=True)

    # Select only the numerical features
    numeric_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

    # Create a pipeline that scales only the selected numeric features
    pipeline = Pipeline(steps=[
        ('column_selector', ColumnTransformer(
            transformers=[
                ('num', MinMaxScaler(), numeric_features)  # Scale the numerical features
            ],
            remainder='passthrough'  # Keep non-numeric features (like 'Country') unchanged
        ))
    ])

    df_scaled = pipeline.fit_transform(df)

    # Ensure the directory exists before saving the pipeline
    os.makedirs(os.path.dirname(pipeline_save_path), exist_ok=True)

    # Save the pipeline
    with open(pipeline_save_path, 'wb') as f:
        pickle.dump(pipeline, f)

    return df_scaled, pipeline

In [30]:
def find_similar_countries(
    ideal_country, 
    df_scaled, 
    pipeline, 
    n_neighbors=10, 
    distance_metric="euclidean",
    feature_importances=None
):
    """
    Find similar countries to 'ideal_country' from 'df_scaled' using a manual distance calculation.
    
    Parameters:
    -----------
    ideal_country : pd.DataFrame or np.ndarray 
        A single-row DataFrame or a 2D array with numeric columns that match those in df_scaled.
    df_scaled : pd.DataFrame
        Scaled DataFrame of countries.
    pipeline : Pipeline
        Fitted pipeline used for scaling and column selection.
    n_neighbors : int
        Number of neighbors (similar countries) to return.
    distance_metric : str
        Distance metric to use to calculate the distance.
    feature_importances : list or np.ndarray
        Per-feature weights.

    Returns:
    --------
    close_countries : pd.DataFrame
        Subset of df with the closest neighbors and a 'country_distance_from_ideal' column.
    """
    # Scale the ideal country
    ideal_country_scaled = pipeline.transform(ideal_country)

    # Apply weighting if provided
    if feature_importances is not None:
        ideal_country_scaled = ideal_country_scaled * feature_importances

    # Compute distances based on the specified metric
    distances = []
    for idx, row in df_scaled.iterrows():
        row_vector = row.values
        if distance_metric == "euclidean":
            dist = np.sqrt(np.sum((row_vector - ideal_country_scaled) ** 2))
        elif distance_metric == "manhattan":
            dist = np.sum(np.abs(row_vector - ideal_country_scaled))
        elif distance_metric == "cosine":
            dot_prod = np.dot(row_vector, ideal_country_scaled)
            denom = (np.linalg.norm(row_vector) * np.linalg.norm(ideal_country_scaled))
            dist = 1.0 - (dot_prod / denom) if denom != 0 else 1.0
        else:
            raise ValueError(f"Unsupported distance metric: {distance_metric}")
        
        distances.append(dist)

    # Sort by distance and pick the top n_neighbors
    nearest_indices = np.argsort(distances)[:n_neighbors]
    close_countries = df_scaled.iloc[nearest_indices].copy()
    close_countries['country_distance_from_ideal'] = [distances[i] for i in nearest_indices]

    return close_countries


In [33]:
df = pd.read_csv("../raw_data/merged_country_level/merged_dataset_with_knn.csv")
df_scaled = scale_dataset(df)
pipeline = pickle.load(open('models/v2_scaler_pipeline.pkl', 'rb'))
ideal_country_array = np.array([[720.0, 13.0, 116.6, 50.6, 75.7]]) 
weights = np.array([1,1,1,1,1])
find_similar_countries(ideal_country_random, df_scaled, pipeline, n_neighbors=155, distance_metric="euclidean", feature_importances=weights)

ValueError: columns are missing: {'Unnamed: 0'}

In [28]:
ideal_country_array.shape

(1, 5)

In [27]:
weights.shape

(5,)